# Industry 4.0 의 중심, BigData

<div align='right'><font size=2 color='gray'>Data Processing Based Python @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, [김진수](bigpycraft@gmail.com)</font></div>
<hr>

# Pandas Basic 5

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np


## 5. Hierarchical indexing
> 계층적 색인
- 축에 대해 다중(둘 이상) 색인 단계를 지정할 수 있도록 해준다.
- 차원이 높은(고차원) 데이터를 낮은 차원의 형식으로 다룰 수 있게 해주는 기능이다.

In [220]:
data = Series(np.random.randn(10),
              index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                     [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])
data

a  1    0.288778
   2    0.622377
   3    2.775512
b  1   -0.003481
   2    0.812371
   3   -0.205898
c  1    0.525355
   2    0.772624
d  2   -0.137761
   3    0.876581
dtype: float64

In [221]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

In [222]:
data['b']

1   -0.003481
2    0.812371
3   -0.205898
dtype: float64

In [223]:
data['b':'c']

b  1   -0.003481
   2    0.812371
   3   -0.205898
c  1    0.525355
   2    0.772624
dtype: float64

In [224]:
data.ix[['b', 'd']]

b  1   -0.003481
   2    0.812371
   3   -0.205898
d  2   -0.137761
   3    0.876581
dtype: float64

In [225]:
data[:, 2]

a    0.622377
b    0.812371
c    0.772624
d   -0.137761
dtype: float64

In [226]:
data.unstack()

1         2         3
a  0.288778  0.622377  2.775512
b -0.003481  0.812371 -0.205898
c  0.525355  0.772624       NaN
d       NaN -0.137761  0.876581

In [227]:
data.unstack().stack()

a  1    0.288778
   2    0.622377
   3    2.775512
b  1   -0.003481
   2    0.812371
   3   -0.205898
c  1    0.525355
   2    0.772624
d  2   -0.137761
   3    0.876581
dtype: float64

> 계층적 색인
- DataFrame에서는 두 축 모두 계층적 색인을 가질수 있다.
- 각 단계는 이름을 가질 수 있고, (문자열이나 어떤 파이썬 객체라도 가능하다)
- 만약 이름이 있다면 콘솔 출력 시에 함께 나타난다. 
#### <font color='blue'> 색인의 이름과 축의 라벨을 혼동하지 말것 !!! </font>

In [228]:
frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'],
                           ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [229]:
frame.index.names   = ['key1',  'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [230]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### <font color='brown'> Reordering and sorting levels </font>
> 계층 순서 바꾸고 정렬하기

In [231]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [232]:
frame.sortlevel(1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [233]:
frame.swaplevel(0, 1).sortlevel(0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

<font color='blue'> [NOTE] 객체가 계층적 색인으로 상위 계층부터 사전식으로 정렬이 되어 있다면 (sortlevel(0)이나 sort_index()의 결과처럼) 데이터를 선택하는 성능이 훨씬 좋아진다.</font>

### <font color='brown'> Summary statistics by level </font>
> 단계별 요약통계
- level 옵션 : 어떤 한 축에 대해 합을 구하고 싶은 단계를 지정할 수 있는 옵션
- 내부적으로는 pandas의 groupby 기능을 이용해서 구현되었다.

In [234]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [235]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [236]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### <font color='brown'> Using a DataFrame's columns </font>
> DataFrame의 칼럼 사용하기
- DataFrame.set_index 함수 : 하나 이상의 칼럼ㅇ르 색인으로 하는 새로운 DataFrame을 생성한다.
- DataFrame.reset_index 함수 : 계층적 샌인 단계가 칼럼으로 이동한다.

In [237]:
frame = DataFrame({'a': range(7), 
                   'b': range(7, 0, -1),
                   'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                   'd': [0, 1, 2, 0, 1, 2, 3]})
frame

a  b    c  d
0  0  7  one  0
1  1  6  one  1
2  2  5  one  2
3  3  4  two  0
4  4  3  two  1
5  5  2  two  2
6  6  1  two  3

In [238]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [239]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [240]:
frame2.reset_index()

c  d  a  b
0  one  0  0  7
1  one  1  1  6
2  one  2  2  5
3  two  0  3  4
4  two  1  4  3
5  two  2  5  2
6  two  3  6  1

<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>